<a href="https://colab.research.google.com/github/rablancoo/BigData/blob/main/Clase_3_20222196002_BigData_ObjectDetection_SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Respositorio usado: https://www.kaggle.com/datasets/mnavaidd/object-detection-and-image-segmentation-dataset?resource=download

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content

!git clone https://github.com/IDEA-Research/Grounded-Segment-Anything

%cd /content/Grounded-Segment-Anything
!pip install -q -r requirements.txt
%cd /content/Grounded-Segment-Anything/GroundingDINO
!pip install -q .
%cd /content/Grounded-Segment-Anything/segment_anything
!pip install -q .
%cd /content/Grounded-Segment-Anything

/content
fatal: destination path 'Grounded-Segment-Anything' already exists and is not an empty directory.
/content/Grounded-Segment-Anything
/content/Grounded-Segment-Anything/GroundingDINO
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
/content/Grounded-Segment-Anything/segment_anything
  Preparing metadata (setup.py) ... done
/content/Grounded-Segment-Anything


In [ ]:
import os
import json
from PIL import Image

# Directorio de imágenes y anotaciones
data_dir = "/content/drive/MyDrive/Colab Notebooks/dataset"  # Reemplaza con la ruta correcta
images_dir = os.path.join(data_dir, "images")
annotations_dir = os.path.join(data_dir, "annotations")
output_dir = os.path.join(data_dir, "labels_yolo")  # Directorio para guardar las etiquetas YOLO

# Crea el directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)


def convert_to_yolo(image_path, annotation_path, output_path):
    with open(annotation_path, 'r') as f:
        data = json.load(f)

    image = Image.open(image_path)
    width, height = image.size

    with open(output_path, 'w') as outfile:
      for key, value in data.items():
        regions = value.get('regions',[])
        for region in regions:
          shape = region['shape_attributes']
          label = region['region_attributes']['name']

          x = shape['x']
          y = shape['y']
          w = shape['width']
          h = shape['height']

          # Calcular coordenadas y dimensiones normalizadas para YOLO
          x_center = ((x + w) / 2) / width
          y_center = ((y + h) / 2) / height
          norm_w = w / width
          norm_h = h / height

          outfile.write(f"{label} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}\n")


# Recorrer cada imagen y anotación
for filename in os.listdir(images_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_name, _ = os.path.splitext(filename)
        annotation_filename = image_name + ".json"  # Suponiendo nombres coincidentes

        image_path = os.path.join(images_dir, filename)
        annotation_path = os.path.join(annotations_dir, annotation_filename)
        output_path = os.path.join(output_dir, image_name + ".txt")

        if os.path.exists(annotation_path):
          try:
              convert_to_yolo(image_path, annotation_path, output_path)
          except Exception as e:
              print(f"Error processing {filename}: {e}")
        else:
            print(f"Annotation file not found for {filename}")

In [ ]:
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

--2024-11-27 00:45:19--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.244.202.25, 18.244.202.73, 18.244.202.103, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.244.202.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth.1’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   122MB/s    in 21s     

2024-11-27 00:45:41 (114 MB/s) - ‘sam_vit_h_4b8939.pth.1’ saved [2564550879/2564550879]



In [ ]:
!pip install --upgrade segment-anything

In [ ]:
import matplotlib.pyplot as plt
from segment_anything import build_sam, SamPredictor, SamAutomaticMaskGenerator
import numpy as np
import torch
import cv2

# Load SAM model
sam_checkpoint = 'sam_vit_h_4b8939.pth'
model_type = "vit_h"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# predictor = SamPredictor(build_sam(checkpoint=sam_checkpoint).to(device))

# Use SamAutomaticMaskGenerator for automatic mask generation with prompts
mask_generator = SamAutomaticMaskGenerator(build_sam(checkpoint=sam_checkpoint).to(device))  # Use mask_generator

/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [ ]:
# prompt: now make a function that segment and mask the picture in the image_path using the yolo labels generated in the code 2

import os
import cv2
import numpy as np
from PIL import Image
from segment_anything import SamAutomaticMaskGenerator, build_sam
import torch

def segment_and_mask_image(image_path, label_path, output_dir):
    """Segments and masks an image based on YOLO labels.

    Args:
        image_path: Path to the input image.
        label_path: Path to the corresponding YOLO label file.
        output_dir: Directory to save the masked images.
    """

    try:
        # Load image
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Load YOLO labels
        with open(label_path, "r") as f:
            labels = f.readlines()

        # Initialize SAM mask generator
        # sam_checkpoint = 'sam_vit_h_4b8939.pth'
        # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # mask_generator = SamAutomaticMaskGenerator(build_sam(checkpoint=sam_checkpoint).to(device))

        # Process each label in the file
        for label in labels:
          label_data = label.strip().split()
          if len(label_data) >= 5:
            class_name = label_data[0]
            x_center, y_center, width, height = map(float, label_data[1:5])

            image_h, image_w, _ = image.shape

            x_min = int((x_center - width / 2) * image_w)
            y_min = int((y_center - height / 2) * image_h)
            x_max = int((x_center + width / 2) * image_w)
            y_max = int((y_center + height / 2) * image_h)

            # Generate masks
            masks = mask_generator.generate(image_rgb)

            # Find the best fitting mask for the bounding box
            best_mask_index = -1
            max_iou = 0
            for i, mask in enumerate(masks):
              mask_bbox = mask["bbox"]
              iou = calculate_iou(mask_bbox, [x_min, y_min, x_max, y_max])
              if iou > max_iou:
                max_iou = iou
                best_mask_index = i

            if best_mask_index != -1:
              best_mask = masks[best_mask_index]["segmentation"]
              masked_image = apply_mask(image, best_mask)
              output_filename = os.path.splitext(os.path.basename(image_path))[0] + f"_{class_name}.png"
              output_path = os.path.join(output_dir, output_filename)
              cv2.imwrite(output_path, masked_image)
              print(f"Saved masked image: {output_path}")

    except Exception as e:
        print(f"Error processing {image_path}: {e}")

def calculate_iou(box1, box2):
  # Calculate intersection area
  x_left = max(box1[0], box2[0])
  y_top = max(box1[1], box2[1])
  x_right = min(box1[2], box2[2])
  y_bottom = min(box1[3], box2[3])

  intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)

  # Calculate union area
  box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
  box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
  union_area = box1_area + box2_area - intersection_area

  if union_area == 0:
    return 0

  # Calculate IoU
  iou = intersection_area / union_area
  return iou

def apply_mask(image, mask):
    masked_image = image.copy()
    masked_image[~mask] = 0  # Set non-mask pixels to black
    return masked_image

In [ ]:
image_dir = "/content/drive/MyDrive/Colab Notebooks/dataset/images"
label_dir = "/content/drive/MyDrive/Colab Notebooks/dataset/labels_yolo"
output_mask_dir = "/content/drive/MyDrive/Colab Notebooks/dataset/masked_images"

# Create the output directory if it doesn't exist
os.makedirs(output_mask_dir, exist_ok=True)

for filename in os.listdir(image_dir):
  if filename.endswith(('.jpg', '.jpeg', '.png')):
    image_name, _ = os.path.splitext(filename)
    image_path = os.path.join(image_dir, filename)
    label_path = os.path.join(label_dir, image_name + ".txt")

    if os.path.exists(label_path):
      segment_and_mask_image(image_path, label_path, output_mask_dir)
    else:
      print(f"Label file not found for {image_path}")

Saved masked image: /content/drive/MyDrive/Colab Notebooks/dataset/masked_images/bag (1)_Handbag.png
Saved masked image: /content/drive/MyDrive/Colab Notebooks/dataset/masked_images/bag (16)_Handbag.png
Saved masked image: /content/drive/MyDrive/Colab Notebooks/dataset/masked_images/bag (14)_Handbag.png
Saved masked image: /content/drive/MyDrive/Colab Notebooks/dataset/masked_images/bag (15)_Handbag.png
Saved masked image: /content/drive/MyDrive/Colab Notebooks/dataset/masked_images/bag (13)_Handbag.png
Saved masked image: /content/drive/MyDrive/Colab Notebooks/dataset/masked_images/bag (12)_Handbag.png
Saved masked image: /content/drive/MyDrive/Colab Notebooks/dataset/masked_images/bag (18)_Handbag.png
Saved masked image: /content/drive/MyDrive/Colab Notebooks/dataset/masked_images/bag (18)_Handbag.png
Saved masked image: /content/drive/MyDrive/Colab Notebooks/dataset/masked_images/bag (18)_Handbag.png


KeyboardInterrupt: 